#Construction d'exercice pour le kalaba

- Transcription de la phonétique TIPA en API UTF8
- Calcul du nombre de morceaux
- Ajout des réponses partielles pour le découpage
 
La partie conclusion est laissée vide

In [3]:
import yaml

In [4]:
dossier="/Copy/Cours/Bordeaux/L1-UE1/Kalaba-14"
serie="Noms"
newMarks=True

In [5]:
from os.path import expanduser
home = expanduser("~")
repertoire=home+dossier
base=repertoire+"/"

In [6]:
from itertools import combinations
import re

In [7]:
marksMoodleOld=[100,90,80,75,70,66.666,60,50,40,33.333,30,25,20,16.666,14.2857,12.5,11.111,10,5,0]
marksMoodleNew=range(101)
if newMarks:
    marksMoodle=marksMoodleNew
else:
    marksMoodle=marksMoodleOld

In [16]:
def noteMoodle(note):
    result=min(marksMoodle, key=lambda x:abs(x-note))
    return "~%%%s%%" % result

In [32]:
def assemblages(analyse):
    '''
    calcul des réponses partielles correspondant à l'analyse
    '''
    chunks = re.findall(ur"(racine([^)]+)x[^+-]+|[+-]|[^-+]+)", analyse)
    morceaux = [x for (x,y) in chunks if not x in ["+","-"]]
    separateurs = [x for (x,y) in chunks if x in ["+","-"]]
    n=len(separateurs)
    formes=[]
    formes.append(analyse)
    if re.search("^[^+]+\+[^+]+\+[^+]+",analyse):
        formes.append("~%%%s%%" % noteMoodle(100-(100*1/n))+analyse.replace("+","-"))
    for i in range(1,n):
        for elements in combinations(range(n),i):
            boolTirets=False
            boolRacine=False
            badRacine=False
            ##########
            #
            # Attention à la division entière : (n-i)/n*100 donne toujours 0
            #
            ##########
            note=100*(n-i)/n
            prefNote=noteMoodle(note)
            forme=""
            for m,morceau in enumerate(morceaux):
                if re.search("racine([^)]+)",morceau):
                    boolRacine=True
                    if not(re.search("[%+-]$",forme)) or (m in elements):
                        badRacine=True
                forme+=morceau
                if not (m in elements) and m<n:
                    forme+=separateurs[m]
                    if separateurs[m] in ["+"]:
                        boolTirets=True
            if not(badRacine):
                formes.append(prefNote+forme)
                if boolTirets:
                    formes.append(noteMoodle(.80*note)+forme.replace('+','-'))
    return "".join(formes)

In [33]:
assemblages('a+test+e-r-ai-t')

'a+test+e-r-ai-t~%~%80%%a-test-e-r-ai-t~%80%atest+e-r-ai-t~%64%atest-e-r-ai-t~%80%a+teste-r-ai-t~%64%a-teste-r-ai-t~%80%a+test+er-ai-t~%64%a-test-er-ai-t~%80%a+test+e-rai-t~%64%a-test-e-rai-t~%80%a+test+e-r-ait~%64%a-test-e-r-ait~%60%ateste-r-ai-t~%60%atest+er-ai-t~%48%atest-er-ai-t~%60%atest+e-rai-t~%48%atest-e-rai-t~%60%atest+e-r-ait~%48%atest-e-r-ait~%60%a+tester-ai-t~%48%a-tester-ai-t~%60%a+teste-rai-t~%48%a-teste-rai-t~%60%a+teste-r-ait~%48%a-teste-r-ait~%60%a+test+erai-t~%48%a-test-erai-t~%60%a+test+er-ait~%48%a-test-er-ait~%60%a+test+e-rait~%48%a-test-e-rait~%40%atester-ai-t~%40%ateste-rai-t~%40%ateste-r-ait~%40%atest+erai-t~%32%atest-erai-t~%40%atest+er-ait~%32%atest-er-ait~%40%atest+e-rait~%32%atest-e-rait~%40%a+testerai-t~%32%a-testerai-t~%40%a+tester-ait~%32%a-tester-ait~%40%a+teste-rait~%32%a-teste-rait~%40%a+test+erait~%32%a-test-erait~%20%atesterai-t~%20%atester-ait~%20%ateste-rait~%20%atest+erait~%16%atest-erait~%20%a+testerait~%16%a-testerait'

In [43]:
def decoupages(analyse):
    morceaux=analyse.split("-")
    n=len(morceaux)-1
    formes=[]
    for i in range(1,n):
        print i,n,i/n
        for element in combinations(range(n),i):
            forme="~%%%s%%" % noteMoodle(100-(100*i/n))
            for m,morceau in enumerate(morceaux):
                forme+=morceau
                if not (m in element) and m<n:
                    forme+="-"
            formes.append(forme)
    return "".join(formes)

In [44]:
def recoder(chaine,table):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    return result

In [45]:
nomCSV=base+"14-Kalaba-"+serie+".csv"
with open(nomCSV, 'r') as file:
    donnees=file.readlines()

In [46]:
nomPhonology=base+"Phonology.yaml"
with open(nomPhonology, 'r') as stream:
    phonology=yaml.load(stream)
tipaIn = unicode(phonology["translations"]["ipa"]["in"])
ipaIn = [ord(char) for char in tipaIn]
ipaOut = unicode(phonology["translations"]["ipa"]["out"])
toipa = dict(zip(ipaIn, ipaOut))

In [47]:
results=[]
for donnee in donnees:
    if donnee.startswith("#"):
        results.append(donnee.strip())
    else:
        champs=donnee.strip().split(";")
        champs[2]=recoder(champs[2],toipa)
        champs[3]=recoder(champs[3],toipa)
#        morceaux=champs[3].split("-")
#        numMorceaux=str(len(morceaux))
        numMorceaux=str(champs[3].count("-")+champs[3].count("+")/2+1)
        champs[3]=assemblages(champs[3])
        champs.insert(3,numMorceaux)
        results.append(";".join(champs))

In [48]:
nomTXT=nomCSV.replace(".csv",".txt")
with open(nomTXT, 'w') as file:
    for result in results:
        file.write(result+"\n")